# Downtown Brooklyn Collisions Study - Advanced Pipeline
This notebook uses UrbanPipeline to analyse collisions with multiple enrichments: injuries and fatalities per intersection.

In [ ]:
import urban_mapper as um
from urban_mapper.pipeline import UrbanPipeline

# Define the pipeline
pipeline = UrbanPipeline([
    ("urban_layer", (
        um.UrbanMapper().urban_layer
        .with_type("streets_intersections")
        .from_place("Downtown Brooklyn, New York City, USA", network_type="drive")
        .with_mapping(
            longitude_column="LONGITUDE",
            latitude_column="LATITUDE",
            output_column="nearest_intersection"
        )
        .build()
    )),
    ("loader", (
        um.UrbanMapper().loader
        .from_file("./NYC_Motor_Vehicle_Collisions_Mar_12_2025.csv")
        .with_columns(longitude_column="LONGITUDE", latitude_column="LATITUDE")
        .build()
    )),
    ("imputer", (
        um.UrbanMapper().imputer
        .with_type("SimpleGeoImputer")
        .on_columns("LONGITUDE", "LATITUDE")
        .build()
    )),
    ("filter", um.UrbanMapper().filter.with_type("BoundingBoxFilter").build()),
    ("enrich_injuries", (
        um.UrbanMapper().enricher
        .with_data(group_by="nearest_intersection", values_from="NUMBER OF PERSONS INJURED")
        .aggregate_by(method="sum", output_column="total_injuries")
        .build()
    )),
    ("enrich_fatalities", (
        um.UrbanMapper().enricher
        .with_data(group_by="nearest_intersection", values_from="NUMBER OF PERSONS KILLED")
        .aggregate_by(method="sum", output_column="total_fatalities")
        .build()
    )),
    ("visualiser", (
        um.UrbanMapper().visual
        .with_type("Interactive")
        .with_style({"tiles": "CartoDB dark_matter"})
        .build()
    ))
])

In [ ]:
# Execute the pipeline
mapped_data, enriched_layer = pipeline.compose_transform()

In [4]:
# Visualize results
fig = pipeline.visualise(["total_injuries", "total_fatalities"])
fig

In [5]:
# Save the pipeline
pipeline.save("./collisions_advanced_pipeline.joblib")